## Assignment 1 - IO

Author: David Henning (worked with Natasha Watkins, Chandni Raja and Jonathan Kowarski)

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
from scipy.optimize import minimize
from scipy import ndimage
from pystout import pystout
from tabulate import tabulate

In [45]:
d = pd.read_csv("PS1_Data/OTC_Data.csv", sep='\t')
d.columns = d.columns.str.replace('_', '')

#Add branded dummy to the dataset
d['branded'] = d['brand']
d.loc[d['branded'] <= 9, 'branded'] = 1
d.loc[d['branded'] > 9, 'branded'] = 0

#Add name and size to the dataset
d['name'] = None
d.loc[d.brand.isin([1, 2, 3]), 'name'] = 'Tylenol'
d.loc[d.brand.isin([4, 5, 6]), 'name'] = 'Advil'
d.loc[d.brand.isin([7, 8, 9]), 'name'] = 'Bayer'
d.loc[d.brand.isin([10, 11]), 'name'] = 'Store brand'
d['size'] = None
d.loc[d.brand.isin([1, 4, 7]), 'size'] = 25
d.loc[d.brand.isin([2, 5, 8, 10]), 'size'] = 50
d.loc[d.brand.isin([3, 6, 9, 11]), 'size'] = 100

In [46]:
d_demographic = pd.read_csv("PS1_Data/OTCDemographics.csv", sep='\t')
d_demographic.columns = d_demographic.columns.str.replace('_', '')

In [47]:
instruments = pd.read_csv("PS1_Data/OTCDataInstruments.csv", sep='\t')
instruments.columns = instruments.columns.str.replace('_', '')

In [48]:
d.sort_values(by=['store', 'week', 'brand'], inplace=True) #Sort dataset

In [49]:
#Try to reproduce summary stat table from the problem set
d_summary = d.copy()
d_summary['sales_store_week'] = d_summary.groupby(['store', 'week'])['sales'].transform('sum')
d_summary['share'] = d_summary['sales'] / d_summary['sales_store_week'] * 0.62
d_summary['price_brand'] = d_summary.groupby(['brand'])['price'].transform('mean')
d_summary['cost_brand'] = d_summary.groupby(['brand'])['cost'].transform('mean')
d_summary['share_brand'] = d_summary.groupby(['brand'])['share'].transform('mean')
d_summary = d_summary[['name', 'size', 'brand', 'share_brand', 'price_brand', 'cost_brand']]
d_summary = d_summary[:11]

### Problem 1

Consider utility function for good $j$ in store-week $t$ for consumer $i$:
    $$ u_{ijt} = X_{jt}\beta + \alpha p_{jt} + \xi_{jt} + \epsilon_{jt} $$
   

###### (1) Estimate this model using OLS with price and promotion as product characteristics

In [50]:
# calculate the shares
df = d[['store', 'week', 'brand', 'prom', 'price', 'cost', 'sales']].copy()
df['sales_store_week'] = df.groupby(['store', 'week'])['sales'].transform('sum')
df['s'] = df['sales'] / df['sales_store_week'] * 0.62 # All of the brands have a total market share of 62%

In [51]:
# Transform variables for regression, recall outside options share is 0.38
s_o = 0.38
df['δ'] = np.log(df['s']) - np.log(s_o)
df['constant'] = 1

In [52]:
# Run OLS regression
x = df.drop(['δ', 'store', 'week', 'brand',  'cost', 'sales', 'sales_store_week', 's',], 1)
y = df['δ']
model1 = sm.OLS(y, x).fit()
model1_α = model1.params['price']

###### (2) Estimate this model using OLS with price and promotion as product characteristics and brand dummies

In [53]:
#generate new dataframe including brand dummies
b_dummies = pd.get_dummies(df['brand'])
df_dummies = df.merge(b_dummies, left_index = True, right_index = True)

In [54]:
x = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost', 'sales', 'sales_store_week', 's',], 1)
y = df_dummies['δ']
model2 = sm.OLS(y, x).fit()
model2_α = model2.params['price']

###### (3) Estimate this model using OLS with price and promotion as product characteristics and store-brand dummies

In [173]:
#generate new dataframe including store-brand dummies
store_brand = df['store'].astype('str') + '_' + df['brand'].astype('str') 
bs_dummies = pd.get_dummies(store_brand)
df_bsdummies = df.merge(bs_dummies, left_index = True, right_index = True)

In [56]:
x = df_bsdummies.drop(['δ', 'store', 's', 'brand', 'week', 'cost', 'sales', 'sales_store_week'], 1)
y = df_bsdummies['δ']
model3 = sm.OLS(y, x).fit()
model3_α = model3.params['price']

In [57]:
#Produce table with regression results from OLS
pystout(models=[model1,model2,model3],
        file='OLS_logit.tex',
        digits=3,
        endog_names=False,
        addrows={'Brand dummies':['No', 'Yes', 'No'], 'Store-brand dummies':['No', 'No', 'Yes']},
        exogvars=['price', 'prom'],
        varlabels={'price':'Price','prom':'Promotion'},
        mgroups={'OLS':[1,3]},
        modstat={'nobs':'Obs','rsquared_adj':'Adj. R\sym{2}'}
        )

###### (4) Estimate the models of 1, 2 and 3 using wholesale cost as an instrument.

In [58]:
# Run IV regression for model 1
x = df.drop(['δ', 'store', 'week', 'brand',  'cost', 'sales', 'sales_store_week', 's',], 1)
z = df.drop(['δ', 'store', 'week', 'brand', 'sales', 'price', 'sales_store_week', 's',], 1)
y = df['δ']
model4 = IV2SLS(y, x, instrument=z).fit()

In [59]:
# Run IV regression for model 2
x = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost', 'sales', 'sales_store_week', 's',], 1)
z = df_dummies.drop(['δ', 'store', 'week', 'brand', 'sales', 'price', 'sales_store_week', 's',], 1)
y = df['δ']
model5 = IV2SLS(y, x, z).fit()

In [60]:
# Run IV regression for model 3
x = df_bsdummies.drop(['δ', 'store', 'week', 'brand',  'cost', 'sales', 'sales_store_week', 's',], 1)
z = df_bsdummies.drop(['δ', 'store', 'week', 'brand', 'price', 'sales', 'sales_store_week', 's',], 1)
y = df['δ']
model6 = IV2SLS(y, x, z).fit()

In [61]:
#Produce table with regression results from IV with costs
pystout(models=[model4,model5,model6],
        file='IV_logit_cost.tex',
        digits=3,
        endog_names=False,
        addrows={'Brand dummies':['No', 'Yes', 'No'], 'Store-brand dummies':['No', 'No', 'Yes']},
        exogvars=['price', 'prom'],
        varlabels={'price':'Price','prom':'Promotion'},
        mgroups={'IV':[1,3]},
        modstat={'nobs':'Obs','rsquared_adj':'Adj. R\sym{2}'}
        )

###### (5) Estimate the models of 1, 2 and 3 using the Hausman instrument (average price in other markets).

In [62]:
#Calculate hausman instrument, and add to all the datasets
grouped = df.groupby(['brand', 'week'])
n = grouped['price'].transform('count')
mean = grouped['price'].transform('mean')
df['price_iv'] = (mean*n - df['price'])/(n-1)
df_dummies['price_iv'] = (mean*n - df['price'])/(n-1)
df_bsdummies['price_iv'] = (mean*n - df['price'])/(n-1)
df.sort_values(by=['brand', 'week', 'store'], inplace=True)

In [63]:
#Run the IV with  the Hausman instrument for model 1
x = df.drop(['δ', 'store', 'week', 'brand',  'cost', 'price_iv', 'sales', 'sales_store_week', 's',], 1)
z = df.drop(['δ', 'store', 'week', 'brand',  'cost', 'price', 'sales', 'sales_store_week', 's',], 1)
y = df['δ']
model7 = IV2SLS(y, x, z).fit()

In [64]:
#Run the IV with  the Hausman instrument for model 2
x = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost', 'price_iv', 'sales', 'sales_store_week', 's',], 1)
z = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost', 'price', 'sales', 'sales_store_week', 's',], 1)
y = df_dummies['δ']
model8 = IV2SLS(y, x, z).fit()

In [66]:
#Run the IV with  the Hausman instrument for model 3
x = df_bsdummies.drop(['δ', 'store', 'week', 'brand',  'cost', 'price_iv', 'sales', 'sales_store_week', 's',], 1)
z = df_bsdummies.drop(['δ', 'store', 'week', 'brand',  'cost', 'price', 'sales', 'sales_store_week', 's',], 1)
y = df_bsdummies['δ']
model9 = IV2SLS(y, x, z).fit()
β_hat = model9.params['price'] #Save coefficient on price for later

In [67]:
#Produce table with regression results from IV with Hausman instruments
pystout(models=[model7,model8,model9],
        file='IV_logit_hausmann.tex',
        digits=3,
        endog_names=False,
        addrows={'Brand dummies':['No', 'Yes', 'No'], 'Store-brand dummies':['No', 'No', 'Yes']},
        exogvars=['price', 'prom'],
        varlabels={'price':'Price','prom':'Promotion'},
        mgroups={'IV':[1,3]},
        modstat={'nobs':'Obs','rsquared_adj':'Adj. R\sym{2}'}
        )

###### (6) Using the analytic formula for elasticity of the logit model, compute the mean own-price elasticities for all brand in the market using the estimates in 1, 2 and 3. Do these results make sense?

In [68]:
#Calculate the own price elasticity for each model, using the elasticity formula for logit
d_summary['model1_η'] = model1_α * d_summary['price_brand'] * (1 - d_summary['share_brand'])
d_summary['model2_η'] = model2_α * d_summary['price_brand'] * (1 - d_summary['share_brand'])
d_summary['model3_η'] = model3_α * d_summary['price_brand'] * (1 - d_summary['share_brand'])

In [69]:
d_price_elast_export = d_summary[['name', 'size', 'model1_η', 'model2_η', 'model3_η']].round(3)
with open("elasticities_logit.tex", "w") as f:
    f.write("\\begin{tabular}{" + "l" + "c" * (len(d_price_elast_export.columns)-1) + "} \hline\hline \n" )
    f.write(" \\textbf{Brand name} & \\textbf{Size} & \\textbf{Model 1} & \\textbf{Model 2} & \\textbf{Model 3}  \\\\\n \hline " )
    for i, row in d_price_elast_export.iterrows():
        f.write(" & ".join([str(x) for x in row.values]) + " \\\\\n")
    f.write("\hline \hline \\\\\n")
    f.write("\\end{tabular}")

## Problem 2 - BLP

Consider utility function for good $j$ in store-week $t$ for consumer $i$:
    $$ u_{ijt} = X_{jt}\beta + \beta_{ib}B_{jt}\text{(Branded Product)} + \alpha_i p_{jt} + \xi_{jt} + \epsilon_{ijt} $$
Where:
$$\beta_{ib} = \sigma_B\nu_i$$
$$\alpha_i = \alpha + \sigma_I I_i$$

Hence $$ u_{ijt} = X_{jt}\beta + \sigma_B\nu_iB_{jt}\text{(Branded Product)} + (\alpha + \sigma_I I_i) p_{jt} + \xi_{jt} + \epsilon_{ijt} $$

Or, rearranging $$ u_{ijt} = X_{jt}\beta + \alpha p_{jt} + \xi_{jt} + \sigma_B\nu_iB_{jt}\text{(Branded Product)} + \sigma_I I_i p_{jt} + \epsilon_{ijt} $$


In [70]:
d_blp = pd.merge(d, d_demographic, on=['store', 'week'])
d_blp['constant'] = 1
d_blp['sales_store_week'] = d_blp.groupby(['store', 'week'])['sales'].transform('sum')
d_blp['s0'] = d_blp['sales'] / d_blp['sales_store_week'] * 0.62 # All of the brands have a total market share of 62%
d_blp['market'] = d_blp.groupby(['store', 'week']).ngroup()
d_blp.sort_values(by=['store', 'week', 'brand'], inplace=True)

In [174]:
#Define function for calculating δ
def iterate_shares(β, σ_B, σ_I, δ, p, v, B, I, X, ids, s):
    max_iter = 10000
    tol = 1e-12
    dist = 1e3      # Initial distance
    j = 0

    while j < max_iter and dist > tol:
        s = np.zeros(I.shape[0])
        
        for i in range(ns):
            numerator = np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)
            denominator =  ndimage.sum(numerator, ids, ids) + 1
            s += numerator / denominator * (1 / ns)
        
        δ_new = δ + np.log(s0) - np.log(s)
        dist = np.linalg.norm(δ_new - δ)
        δ = δ_new
        j += 1
            
    return δ

In [72]:
#Define the gmm function
def gmm(params, p, v, B, I, X, Z, ids, s0):
    β = params[:-2]
    σ_B = params[-2]
    σ_I = params[-1]
    
    δ_init = X @ β #Initial δ
    
    δ = iterate_shares(β, σ_B, σ_I, δ_init, p, v, B, I, X, ids, s0)
    ξ = δ - X @ β
    Q = Z.T @ ξ @ np.linalg.inv(Z.T @ Z) @ Z.T @ ξ
    print(Q)
    
    return Q

In [73]:
# Put the data in the necessary format
X = d_blp[['price', 'constant', 'prom']].copy() #Include price, constant and promotions
brand_dummies = pd.get_dummies(d_blp['brand']) #get brand dummies
X = pd.concat([X, brand_dummies], axis=1) # add brand dummies to X
X = X.values #turn it into an array
p = d_blp['price'].values
B = d_blp['branded'].values
I = d_blp.iloc[:, d_blp.columns.str.startswith('hhincome')].values
Z = instruments.iloc[:, 3:].values
ids = d_blp['market'].values
s0 = d_blp['s0'].values

#Setting the frequencies
ns = I.shape[1]

#Set our random draws from the normal distribution
np.random.seed(1)
v = np.random.normal(size=ns)

In [32]:
#Set the starting values for parameters
params = np.ones(X.shape[1] + 2) # all to 1, except for..
params[0] = -1.5 #price coefficient is set to -1.5

bounds = [[None, None]] * len(params)
bounds[0] = [None, 0]  # Price coefficient must be negative

In [33]:
#Minimize gmm
results = minimize(gmm, params, args=(p, v, B, I, X, Z, ids, s0), 
                   method='L-BFGS-B',
                   bounds=bounds,
                   tol=1e-5,
                   options={'disp' : True})

87888519.88131578
87888519.70675772
87888519.9154053
87888519.88495928
87888519.8837772
87888519.88481718
87888519.8862721
87888519.88345313
87888519.88497826
87888519.88707906
87888519.88324326
87888519.88390844
87888519.88412353
87888519.88269708
87888519.88447821
87888519.87942535
87888521.76379874
476241.573027822
476241.58561681653
476241.5706441677
476241.572753335
476241.5728793066
476241.57276867644
476241.57261335745
476241.5728871569
476241.5727750296
476241.57259761484
476241.5728994588
476241.57286072324
476241.57275433943
476241.57297161163
476241.57291366014
476241.5734358409
476241.4392059514
19135.88008876711
19135.880366679852
19135.88010482521
19135.880070243496
19135.880109638383
19135.880075849447
19135.880025259306
19135.88009521589
19135.880092113373
19135.880065573736
19135.880093634172
19135.88010074447
19135.880015278846
19135.880127148346
19135.88019203995
19135.880113616426
19135.87706115235
18902.37181415747
18902.371806722855
18902.371885822977
18902.371801

In [175]:
results.x

array([-1.66381975,  0.02549763,  1.16766022,  0.66632234,  1.09551766,
        1.74134164,  0.8669196 ,  0.87418101,  1.18264668,  0.89260881,
        0.78263597,  1.92305578,  0.45497064, -0.4548423 ,  0.78311546,
        0.0957516 ])

In [176]:
table = [["$\\alpha$",results.x[0]],["$\\sigma_I$",results.x[-1]],["$\\sigma_B$",results.x[-2]], ['GMM criterion value',results.fun]]
with open('blp_estimates.tex', 'w') as f:
    print(tabulate(table, headers=['Parameters', 'Point estimates'], colalign=('left','center'), tablefmt='latex_raw', floatfmt='.3f'), file=f)

###### (2) Calculate the own and cross-price elasticity for store 9 in week 10

In [238]:
#Only select store 9, week 10
d_blp_elastic = d_blp[(d_blp['store'] == 9) & (d_blp['week'] ==10)].copy()
J = d_blp_elastic.shape[0]

In [239]:
#Reassign values
X = d_blp_elastic[['price', 'constant', 'prom']].copy() #Include price, constant and promotions
brand_dummies = pd.get_dummies(d_blp_elastic['brand']) #get brand dummies
X = pd.concat([X, brand_dummies], axis=1) # add brand dummies to X
X = X.values
name = d_blp_elastic['name'].values
size = d_blp_elastic['size'].values
cost = d_blp_elastic['cost'].values
p = d_blp_elastic['price'].values
B = d_blp_elastic['branded'].values
I = d_blp_elastic.iloc[:, d_blp_elastic.columns.str.startswith('hhincome')].values
s0 = d_blp_elastic['s0'].values

In [240]:
#Save estimation results
θ = results.x
α = θ[0]
β = θ[:-2]
σ_B = θ[-2]
σ_I = θ[-1]
δ = X @ β

In [241]:
def get_η(δ, p, v, B, I, X, s0, σ_B, σ_I):

        η = np.zeros((J, J))
        
        for j in range(J):
            s_sum = 0
            for i in range(ns):
                denominator =  np.sum(np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)) + 1
                s_ij = np.exp(δ[j] + σ_B * v[i] * B[j] + σ_I * I[j, i] * p[j]) / denominator
                s_sum += (s_ij - s_ij**2) * (α + σ_I * I[j, i]) * (1 / ns)
            η[j,j] = (p[j] / s0[j]) * s_sum
            
        for j in range(J):
            for k in range(J):
                if j != k:
                    s_sum = 0
                    for i in range(ns):
                        denominator =  np.sum(np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)) + 1
                        s_ij = np.exp(δ[j] + σ_B * v[i] * B[j] + σ_I * I[j, i] * p[j]) / denominator
                        s_ik = np.exp(δ[k] + σ_B * v[i] * B[k] + σ_I * I[k, i] * p[k]) / denominator
                        s_sum += s_ij * s_ik * (α + σ_I * I[j, i]) * (1 / ns)
                    η[j,k] = - (p[k] / s0[j]) * s_sum
            
        return η

In [242]:
η = get_η(δ, p, v, B, I, X, s0, σ_B, σ_I)
row = np.arange(1, J + 1)

In [243]:
with open('elasticity_blp.tex', 'w') as f:
    print(tabulate(η,  showindex=row, headers=range(1, J + 1), numalign ='center', tablefmt='latex_raw', floatfmt=('.0f','.3f','.3f','.3f','.3f','.3f','.3f','.3f','.3f','.3f','.3f','.3f')), file=f)

In [244]:
def get_η_logit(δ, p, v, B, I, X, s0, σ_B, σ_I):
        
        σ_B = 0
        σ_I = 0
        η = np.zeros((J, J))
        
        for j in range(J):
            s_sum = 0
            for i in range(ns):
                denominator =  np.sum(np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)) + 1
                s_ij = np.exp(δ[j] + σ_B * v[i] * B[j] + σ_I * I[j, i] * p[j]) / denominator
                s_sum += (s_ij - s_ij**2) * (α + σ_I * I[j, i]) * (1 / ns)
            η[j,j] = (p[j] / s0[j]) * s_sum
            
        for j in range(J):
            for k in range(J):
                if j != k:
                    s_sum = 0
                    for i in range(ns):
                        denominator =  np.sum(np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)) + 1
                        s_ij = np.exp(δ[j] + σ_B * v[i] * B[j] + σ_I * I[j, i] * p[j]) / denominator
                        s_ik = np.exp(δ[k] + σ_B * v[i] * B[k] + σ_I * I[k, i] * p[k]) / denominator
                        s_sum += s_ij * s_ik * (α + σ_I * I[j, i]) * (1 / ns)
                    η[j,k] = - (p[k] / s0[j]) * s_sum
            
        return η

In [245]:
η_logit = get_η_logit(δ, p, v, B, I, X, s0, σ_B, σ_I)
row = np.arange(1, J + 1).round(0)

In [246]:
with open('elasticity_logit.tex', 'w') as f:
    print(tabulate(η_logit,  showindex=row, headers=range(1, J + 1), numalign ='center', tablefmt='latex_raw', floatfmt=('.0f','.3f','.3f','.3f','.3f','.3f','.3f','.3f','.3f','.3f','.3f','.3f')), file=f)

###### (3) Back out marginal costs for store 9, week 10

In [247]:
def get_partials(δ, p, v, B, I, X, s0, σ_B, σ_I):

        partials = np.zeros((J, J))
        
        for j in range(J):
            s_sum = 0
            for i in range(ns):
                denominator =  np.sum(np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)) + 1
                s_ij = np.exp(δ[j] + σ_B * v[i] * B[j] + σ_I * I[j, i] * p[j]) / denominator
                s_sum += (s_ij - s_ij**2) * (α + σ_I * I[j, i]) * (1 / ns)
            partials[j,j] = s_sum
            
        for j in range(J):
            for k in range(J):
                if j != k:
                    s_sum = 0
                    for i in range(ns):
                        denominator =  np.sum(np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)) + 1
                        s_ij = np.exp(δ[j] + σ_B * v[i] * B[j] + σ_I * I[j, i] * p[j]) / denominator
                        s_ik = np.exp(δ[k] + σ_B * v[i] * B[k] + σ_I * I[k, i] * p[k]) / denominator
                        s_sum += s_ij * s_ik * (α + σ_I * I[j, i]) * (1 / ns)
                    partials[j,k] = - s_sum
            
        return partials

In [248]:
Ω_pre = np.zeros((J, J))
Ω_pre[:3, :3] = 1        # Fill in Tylenol ownership
Ω_pre[3:6, 3:6] = 1      # Fill in Advil ownership
Ω_pre[6:9, 6:9] = 1      # Fill in Bayer ownership
Ω_pre[9:, 9:] = 1        # Fill in store brand ownership

In [251]:
partials = get_partials(δ, p, v, B, I, X, s0, σ_B, σ_I)
mc_blp = p + np.linalg.inv(Ω_pre * partials) @ s0
diff  = cost - mc_blp

In [252]:
table = np.array([name, size, cost, mc_blp, diff]).T
with open('mc.tex', 'w') as f:
    print(tabulate(table, headers=['Brand', 'Size', 'Wholesale costs', 'Estimated marginal costs', 'Difference'], numalign ='center', tablefmt='latex_raw', floatfmt=('.0f','.2f','.2f','.2f','.2f')), file=f)

## Problem 3 - merger analysis

In [195]:
def get_partials_logit(α, s0):

        partials = np.zeros((J, J))
        
        for j in range(J):
            partials[j,j] = (s0[j] - s0[j]**2) * α
            
        for j in range(J):
            for k in range(J):
                if j != k:
                    partials[j,k] = - s0[j] * s0[k] * α 
            
        return partials

In [253]:
partials_logit = get_partials_logit(β_hat, s0)
mc_logit =  p + np.linalg.inv(Ω_pre * partials_logit) @ s0

In [254]:
Ω_post = np.zeros((J, J))
Ω_post[:9, :9] = 1  
Ω_post[9:, 9:] = 1  
Ω_post

p_logit = mc_logit - np.linalg.inv(Ω_post * partials_logit) @ s0

In [255]:
table = np.array([name, size, p, p_logit]).T
with open('merger_prices_logit.tex', 'w') as f:
    print(tabulate(table, headers=['Brand', 'Size', 'Pre-merger prices', 'Post-merger prices'], numalign ='center', tablefmt='latex_raw', floatfmt=('.0f','.2f','.2f','.2f')), file=f)

###### (3) Conduct the merger analysis for BLP

In [257]:
p_blp = mc_blp - np.linalg.inv(Ω_post * partials) @ s0 #calculate new prices
p_blp

array([ 5.19512867,  6.79046967,  8.31007528,  5.12219297,  7.62842908,
       10.76426021,  4.94682047,  5.5918192 ,  6.23562211,  1.69      ,
        4.49      ])

In [258]:
table = np.array([name, size, p, p_blp]).T
with open('merger_prices_blp.tex', 'w') as f:
    print(tabulate(table, headers=['Brand', 'Size', 'Pre-merger prices', 'Post-merger prices'], numalign ='center', tablefmt='latex_raw', floatfmt=('.0f','.2f','.2f','.2f')), file=f)